<a href="https://colab.research.google.com/github/Alton1998/ausa-blood-pressure-notebook/blob/main/BP_model_tensoflow_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflowjs
!pip install TensorFlow==2.15.0
!pip install tensorflow-decision-forests==1.8.1

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.16.0
    Uninstalling tf_keras-2.16.0:
      Successfully uninstalled tf_keras-2.16.0


In [ ]:
import numpy as np
import tensorflowjs as tfjs  # Using tfjs_converter for newer versions
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Multiply,
    Input,
    BatchNormalization,
    Activation,
    LSTM,
)
from tensorflow.keras.callbacks import EarlyStopping

def generate_fake_data(number_of_patients=365):
    X_context = []
    X_time_series = []
    y = []
    for _ in range(number_of_patients):
        age = np.random.randint(0, 101)
        sex = np.random.randint(0, 2)
        bmi = np.random.randint(30, 40)
        systolic_bp = np.random.randint(80, 120, size=14)  # 14 readings
        diastolic_bp = np.random.randint(70, 80, size=14)
        future_systolic_bp = np.random.randint(80, 120, size=7)
        future_diastolic_bp = np.random.randint(70, 80, size=7)
        result = np.concatenate((future_systolic_bp, future_diastolic_bp), axis=0)
        X_context.append([age, sex, bmi])
        X_time_series.append([systolic_bp, diastolic_bp])
        y.append([result])
    return np.array(X_context, dtype=float), np.array(X_time_series, dtype=float), np.array(y, dtype=float)

clear_session()

X_context, X_time_series, y = generate_fake_data()

X_time_series = X_time_series.reshape(365, 14, 2)  # Reshape for 14 timesteps, 2 features
y = y.reshape(365, 14, 1)

input_context_layer = Input(shape=(3,))
context_layer = Dense(3, activation="relu")(input_context_layer)
batch_norm_layer = BatchNormalization()(context_layer)
sigmoid_layer = Activation(activation="sigmoid")(batch_norm_layer)

input_time_series_layer = Input(shape=(14, 2))  # Input for 14 timesteps
bp_sequence_layer = LSTM(3, activation="tanh")(input_time_series_layer)
multiply_layer = Multiply()([sigmoid_layer, bp_sequence_layer])
final_layer = Dense(14)(multiply_layer)  # Output layer for 14 predictions

model = Model(inputs=[input_context_layer, input_time_series_layer], outputs=final_layer)
model.compile(optimizer="adam", loss="mse")
early_stop = EarlyStopping(monitor="val_loss", patience=5)
model.fit(x=[X_context, X_time_series], y=y, batch_size=32, epochs=100, validation_split=0.2, callbacks=[early_stop])
model.save("bp.keras")

tfjs.converters.save_keras_model(model, "bp")


Epoch 1/100
10/10 [==============================] - 6s 50ms/step - loss: 7797.0020 - val_loss: 7768.9761
Epoch 2/100
10/10 [==============================] - 0s 12ms/step - loss: 7795.2622 - val_loss: 7767.2388
Epoch 3/100
10/10 [==============================] - 0s 11ms/step - loss: 7793.5229 - val_loss: 7765.5020
Epoch 4/100
10/10 [==============================] - 0s 8ms/step - loss: 7791.7832 - val_loss: 7763.7671
Epoch 5/100
10/10 [==============================] - 0s 10ms/step - loss: 7790.0464 - val_loss: 7762.0317
Epoch 6/100
10/10 [==============================] - 0s 8ms/step - loss: 7788.3066 - val_loss: 7760.2964
Epoch 7/100
10/10 [==============================] - 0s 10ms/step - loss: 7786.5684 - val_loss: 7758.5615
Epoch 8/100
10/10 [==============================] - 0s 8ms/step - loss: 7784.8320 - val_loss: 7756.8262
Epoch 9/100
10/10 [==============================] - 0s 10ms/step - loss: 7783.0942 - val_loss: 7755.0923
Epoch 10/100
10/10 [=============================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!zip -r /content/bp.zip /content/bp

updating: content/bp/ (stored 0%)
updating: content/bp/model.json (deflated 79%)
updating: content/bp/group1-shard1of1.bin (deflated 0%)


In [6]:
model.predict([np.array([[27.0, 0.0, 33.0]]), np.array([[
    [125.0, 78.0],
    [135.0, 85.0],
    [118.0, 76.0],
    [132.0, 88.0],
    [140.0, 92.0],
    [128.0, 82.0],
    [130.0, 84.0],
    [119.0, 75.0],
    [137.0, 87.0],
    [134.0, 90.0],
    [122.0, 80.0],
    [136.0, 85.0],
    [125.0, 78.0],
    [139.0, 89.0],
  ]])])

1/1 [==============================] - 0s 234ms/step


array([[0.9971407 , 0.99705845, 0.9969328 , 0.9965869 , 0.99745864,
        0.99732715, 0.99746954, 0.99718285, 0.99736863, 0.9968789 ,
        0.99705374, 0.9973343 , 0.997262  , 0.9971345 ]], dtype=float32)

In [5]:
X_context[0]

array([ 4.,  1., 36.])